In [12]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [13]:
!pip install nvcc4jupyter

In [14]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [15]:
%%cuda
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [16]:
%%cuda
#include <stdio.h>
#include <time.h>
#include "cuda_runtime.h"

const int MAXN = 16;
const int INF = 1e9;
const int MIN_EDGE_WEIGHT = 1;
const int MAX_EDGE_WEIGHT = 10;

const int N = 10;

long long factorial[MAXN+1];

/////////////////// Host Functions ///////////////////

__host__ int random(int l, int r) {
  return l + rand()%(r-l+1);
}

__host__ void precompute_factorial() {
	factorial[0] = 1;

	for(int i=1;i<=MAXN;i++)
	{
		factorial[i] = i * factorial[i-1];
	}
}

__host__ void assign_edge_weights(int matrix[N][N]) {
	for (int i = 0 ; i < N ; i++) {
		for (int j = i+1 ; j < N ; j++) {
			matrix[i][j] = random(MIN_EDGE_WEIGHT,MAX_EDGE_WEIGHT);
			matrix[j][i] = matrix[i][j];
		}
		matrix[i][i] = 0;
	}
}


__host__ void print_matrix(int matrix[N][N]) {
	for(int i=0; i<N; i++) {
		for(int j=0; j<N; j++) {
			// cout << matrix[i][j] << " ";
            printf("%d ", matrix[i][j]);
		}
		printf("\n");
	}
}

/////////////////// Device Functions ///////////////////

__device__ void swap(int &a, int &b) {
	int temp = a;
	a = b;
	b = temp;
}

__device__ long long fact(int n) {
	long long ans = 1;
	for(int i=1;i<=n;i++) {
		ans *= i;
	}
	return ans;
}

__device__ bool nxt_permutation(int *arr, int n) {
	bool nxt_permutation_possible = false;

	int fi = -1;

	for(int i=n-2;i>=0;i--) {
		if(arr[i+1] > arr[i]) {
			nxt_permutation_possible = true;
			fi = i;
			break;
		}
	}

	if(!nxt_permutation_possible)return false;

	int next_greater_ele = arr[fi+1], next_greater_ele_ind = fi+1;

	for(int i=fi+2;i<n;i++) {
		if(arr[i] > arr[fi] && arr[i] < next_greater_ele) {
			next_greater_ele = arr[i];
			next_greater_ele_ind = i;
		}
	}

	swap(arr[fi],arr[next_greater_ele_ind]);

	//Reverse
	int li = fi+1, ri = n-1;
	while(li < ri) {
		swap(arr[li],arr[ri]);
		li++;
		ri--;
	}

	return true;
}

//Input array should be sorted
__device__ bool nth_permutation(int *arr, int arrsize, long long n) {
	if(n>fact(arrsize))return false;

    // Assuming arrSize = N+1
	// bool taken[arrsize];
    bool taken[N+1];

	for(int i=0; i<arrsize; i++) taken[i] = false;

	int *ans = new int[arrsize];

	for(int i=0; i<arrsize; i++) {
		int cn = 1;
		long long cval = fact(arrsize-1-i);

		while(cval<n) {
			cn++;
			cval=(long long)cn*cval;
			cval=(long long)cval/(cn-1);
		}

		long long pval = cval*(cn-1)/cn;
		n -= pval;

		for(int j=0; j<arrsize; j++) {
			if(!taken[j]) {
				cn--;
				if(cn==0) {
					ans[i] = arr[j];
					taken[j] = true;
					break;
				}
			}
		}
	}

	for(int i=0; i<arrsize; i++) {
		arr[i] = ans[i];
	}
	free(ans);
	return true;
}

__device__ int find_path_cost(int* matrix, int* arr, int arrsize, int n) {
	int cost = 0;
	for(int i=1; i<arrsize; i++) {
        int to = arr[i];
        int from = arr[i-1];
		cost += matrix[to*n + from];
	}
	return cost;
}

/////////////////// Global Functions ///////////////////

__global__ void tsp_cuda(int* matrix, int* path, long long* factorials) {

    __shared__ int thread_optimal_values[(N-1)*(N-2)];
    __shared__ int* thread_optimal_paths[(N-1)*(N-2)];

    int thread = threadIdx.x + blockIdx.x * blockDim.x;
    thread_optimal_values[thread] = INF;
    thread_optimal_paths[thread] = new int[N+1];

    long long iter_per_thread = factorials[N-1] / ((N-1)*(N-2));

    int arr[N-1];
    for (int i = 1; i < N; i++) arr[i-1] = path[i];
    nth_permutation(arr, N-1, (thread * iter_per_thread) + 1);

    long long iter = 0;
    do {

        int temp_path[N+1];
        temp_path[0] = 0;
        for (int i = 1; i < N; i++) temp_path[i] = arr[i-1];
        temp_path[N] = 0;


        int val = find_path_cost(matrix, temp_path, N+1, N);
        // printf("Thread: %d\tCost: %d\n", thread, val);

        if(val < thread_optimal_values[thread])
		{
			thread_optimal_values[thread] = val;
            for (int i = 0; i < N+1; i++) thread_optimal_paths[thread][i] = temp_path[i];
		}

        iter++;
        nxt_permutation(arr, N-1);

    } while (iter < iter_per_thread);

    __syncthreads();

    if (thread == 0) {
        int optimal_cost = INF;
        for (int i = 0; i < (N-1)*(N-2); i++) {

            if (thread_optimal_values[i] < optimal_cost) {
                optimal_cost = thread_optimal_values[i];
                for (int j = 0; j < N+1; j++) {
                    path[j] = thread_optimal_paths[i][j];
                }
            }

        }
    }
}


//////////////////////////////////////////////////////////////

int main(int argc, char **argv) {
    precompute_factorial();
	srand(time(NULL));

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int matrix[N][N], path[N+1];

    path[0] = 0;
    path[N] = 0;
    for (int i = 1; i < N; i++) path[i] = i;

    assign_edge_weights(matrix);

    print_matrix(matrix);

    int *dev_matrix, *dev_path;
    long long *dev_factorial;
    int mat_size = N*N*sizeof(int);
    int path_size = (N+1)*sizeof(int);
    int factorial_size = (MAXN+1)*sizeof(long long);

    cudaMalloc((void **)&dev_matrix, mat_size);
    cudaMalloc((void **)&dev_path, path_size);
    cudaMalloc((void **)&dev_factorial, factorial_size);

    cudaEventRecord(start);

    // Copy inputs from host to device
    cudaMemcpy(dev_matrix, matrix, mat_size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_path, path, path_size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_factorial, factorial, factorial_size, cudaMemcpyHostToDevice);

    // Launch the TSP kernel
    tsp_cuda<<<1, (N-1)*(N-2)>>>(dev_matrix, dev_path, dev_factorial);

    // Copy result from device to host
    cudaMemcpy(path, dev_path, path_size, cudaMemcpyDeviceToHost);
	cudaDeviceSynchronize();
    cudaEventRecord(stop);

    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // printing the minimum cost path
    printf("Minimum Cost Path: ");
    for (int i = 0; i < N+1; i++) {
        printf("%d ", path[i]);
    }
    printf("\n");

    // printing the minimum cost path
    int cost = 0;
    for(int i=1; i<N+1; i++) {
        cost += matrix[path[i]][path[i-1]];
    }
    printf("Path cost: %d \n", cost);

    // printing the run-time
    printf("Time taken: %f s\n", milliseconds*0.001);

    cudaFree(dev_matrix);
    cudaFree(dev_path);
}


0 2 10 5 6 7 6 2 8 10 
2 0 7 1 10 1 7 10 5 8 
10 7 0 1 9 4 6 7 10 1 
5 1 1 0 9 6 1 4 8 8 
6 10 9 9 0 8 1 7 5 6 
7 1 4 6 8 0 5 10 10 2 
6 7 6 1 1 5 0 9 6 2 
2 10 7 4 7 10 9 0 10 8 
8 5 10 8 5 10 6 10 0 8 
10 8 1 8 6 2 2 8 8 0 
Minimum Cost Path: 0 1 5 9 2 3 6 4 8 7 0 
Path cost: 26 
Time taken: 0.006664 s



In [25]:
%%cuda
#include <stdio.h>
#include <time.h>
#include "cuda_runtime.h"

const int MAXN = 16;
const int INF = 1e9;
const int MIN_EDGE_WEIGHT = 1;
const int MAX_EDGE_WEIGHT = 10;

// Initialize factorial array for maximum possible nodes
long long factorial[MAXN+1];

/////////////////// Host Functions ///////////////////

__host__ int random(int l, int r) {
  return l + rand() % (r - l + 1);
}

__host__ void precompute_factorial() {
	factorial[0] = 1;
	for(int i = 1; i <= MAXN; i++) {
		factorial[i] = i * factorial[i - 1];
	}
}

__host__ void assign_edge_weights(int* matrix, int n) {
	for (int i = 0; i < n; i++) {
		for (int j = i + 1; j < n; j++) {
			matrix[i * n + j] = random(MIN_EDGE_WEIGHT, MAX_EDGE_WEIGHT);
			matrix[j * n + i] = matrix[i * n + j];
		}
		matrix[i * n + i] = 0;
	}
}

/////////////////// Device Functions ///////////////////

__device__ void swap(int &a, int &b) {
	int temp = a;
	a = b;
	b = temp;
}

__device__ bool nth_permutation(int *arr, int arrsize, long long n, long long* dev_factorial) {
	if(n > dev_factorial[arrsize]) return false;

	bool taken[MAXN];
	for(int i = 0; i < arrsize; i++) taken[i] = false;

	int *ans = new int[arrsize];

	for(int i = 0; i < arrsize; i++) {
		int cn = 1;
		long long cval = dev_factorial[arrsize - 1 - i];

		while(cval < n) {
			cn++;
			cval = (long long)cn * cval;
			cval = (long long)cval / (cn - 1);
		}

		long long pval = cval * (cn - 1) / cn;
		n -= pval;

		for(int j = 0; j < arrsize; j++) {
			if(!taken[j]) {
				cn--;
				if(cn == 0) {
					ans[i] = arr[j];
					taken[j] = true;
					break;
				}
			}
		}
	}

	for(int i = 0; i < arrsize; i++) {
		arr[i] = ans[i];
	}
	free(ans);
	return true;
}

__device__ int find_path_cost(int* matrix, int* arr, int arrsize, int n) {
	int cost = 0;
	for(int i = 1; i < arrsize; i++) {
		cost += matrix[arr[i] * n + arr[i - 1]];
	}
	return cost;
}

/////////////////// Global Functions ///////////////////

__global__ void tsp_cuda(int* matrix, int* path, long long* dev_factorial, int n) {
    __shared__ int thread_optimal_values[MAXN*(MAXN-1)];
    __shared__ int* thread_optimal_paths[MAXN*(MAXN-1)];

    int thread = threadIdx.x + blockIdx.x * blockDim.x;
    thread_optimal_values[thread] = INF;
    thread_optimal_paths[thread] = new int[n+1];

    long long iter_per_thread = dev_factorial[n-1] / ((n-1)*(n-2));

    int arr[MAXN-1];
    for (int i = 1; i < n; i++) arr[i-1] = path[i];
    nth_permutation(arr, n-1, (thread * iter_per_thread) + 1, dev_factorial);

    long long iter = 0;
    do {
        int temp_path[MAXN+1];
        temp_path[0] = 0;
        for (int i = 1; i < n; i++) temp_path[i] = arr[i-1];
        temp_path[n] = 0;

        int val = find_path_cost(matrix, temp_path, n+1, n);

        if(val < thread_optimal_values[thread]) {
			thread_optimal_values[thread] = val;
            for (int i = 0; i < n+1; i++) thread_optimal_paths[thread][i] = temp_path[i];
		}

        iter++;
        __syncthreads();
    } while (iter < iter_per_thread);

    if (thread == 0) {
        int optimal_cost = INF;
        for (int i = 0; i < (n-1)*(n-2); i++) {
            if (thread_optimal_values[i] < optimal_cost) {
                optimal_cost = thread_optimal_values[i];
                for (int j = 0; j < n+1; j++) {
                    path[j] = thread_optimal_paths[i][j];
                }
            }
        }
    }
}

//////////////////////////////////////////////////////////////

int main(int argc, char **argv) {
	precompute_factorial();
	srand(time(NULL));

	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

    // Loop through each value of N from 3 to 14
    for (int n = 3; n <= 14; n++) {
        int matrix[MAXN*MAXN], path[MAXN+1];

        // Initialize path array (start at 0, path back to 0)
        path[0] = 0;
        path[n] = 0;
        for (int i = 1; i < n; i++) path[i] = i;

        assign_edge_weights(matrix, n);

        int *dev_matrix, *dev_path;
        long long *dev_factorial;
        int mat_size = n * n * sizeof(int);
        int path_size = (n + 1) * sizeof(int);
        int factorial_size = (MAXN + 1) * sizeof(long long);

        cudaMalloc((void **)&dev_matrix, mat_size);
        cudaMalloc((void **)&dev_path, path_size);
        cudaMalloc((void **)&dev_factorial, factorial_size);

        cudaMemcpy(dev_matrix, matrix, mat_size, cudaMemcpyHostToDevice);
        cudaMemcpy(dev_path, path, path_size, cudaMemcpyHostToDevice);
        cudaMemcpy(dev_factorial, factorial, factorial_size, cudaMemcpyHostToDevice);

        cudaEventRecord(start);

        // Launch the TSP kernel
        tsp_cuda<<<1, (n-1)*(n-2)>>>(dev_matrix, dev_path, dev_factorial, n);

        cudaMemcpy(path, dev_path, path_size, cudaMemcpyDeviceToHost);

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float milliseconds = 0;
        cudaEventElapsedTime(&milliseconds, start, stop);

        // Output the time taken for the current number of nodes
        printf("N = %d, Time taken: %f seconds\n", n, milliseconds * 0.001);

        cudaFree(dev_matrix);
        cudaFree(dev_path);
        cudaFree(dev_factorial);
    }

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

	return 0;
}


N = 3, Time taken: 0.001730 seconds
N = 4, Time taken: 0.000130 seconds
N = 5, Time taken: 0.000199 seconds
N = 6, Time taken: 0.000288 seconds
N = 7, Time taken: 0.000463 seconds
N = 8, Time taken: 0.000648 seconds
N = 9, Time taken: 0.001642 seconds
N = 10, Time taken: 0.009666 seconds
N = 11, Time taken: 0.077109 seconds
N = 12, Time taken: 0.374265 seconds
N = 13, Time taken: 2.976513 seconds
N = 14, Time taken: 35.923082 seconds

